# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
city_df = pd.read_csv('../WeatherPy/clean_weather.csv')
city_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date
0,Ilulissat,GL,69.2167,-51.1000,37.40,52,75,4.61,1619264423
1,Vanavara,RU,60.3400,102.2797,34.81,94,100,1.66,1619264423
2,Ushuaia,AR,-54.8000,-68.3000,44.60,81,75,4.61,1619264340
3,Camaná,PE,-16.6228,-72.7111,66.33,84,62,7.20,1619264424
4,Vaini,TO,-21.2000,-175.2000,73.40,94,20,2.30,1619264424
...,...,...,...,...,...,...,...,...,...
557,Havre-St-Pierre,CA,50.2334,-63.5986,32.00,86,1,6.91,1619264527
558,São Félix do Xingu,BR,-6.6447,-51.9950,73.83,97,84,0.27,1619264527
559,Mountain Home,US,43.1330,-115.6912,50.00,37,1,16.11,1619264527
560,Saint Augustine,US,29.8947,-81.3145,68.50,78,1,12.66,1619264528


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
#configure gmaps key
gmaps.configure(api_key=g_key)

# Setting the parameters for the map

humidity = city_df["Humidity"].astype(float)
maxhum = humidity.max()
locations = city_df[["Latitude", "Longitude"]]


In [34]:
# Generating the heat map.

fig = gmaps.figure()

# Describing the heat map
heat = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhum,point_radius=3)

fig.add_layer(heat)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
# Find cities ideal for vacation 
narrowed_city_df = city_df.loc[(city_df["Temperature"] > 70) & (city_df["Temperature"] < 80) & (city_df["Cloudiness"] == 0), :]

# Drop cities outside the ideal condition
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date
0,Cape Town,ZA,-33.9258,18.4232,71.10,56,0,6.91,1619264168
1,São João da Barra,BR,-21.6403,-41.0511,73.40,83,0,4.61,1619264254
2,East London,ZA,-33.0153,27.9116,75.20,53,0,10.36,1619264449
3,Carnarvon,AU,-24.8667,113.6333,75.20,73,0,5.75,1619264253
4,Arraial do Cabo,BR,-22.9661,-42.0278,74.17,83,0,10.36,1619264326
5,Tiarei,PF,-17.5333,-149.3333,77.00,78,0,4.61,1619264469
6,Geraldton,AU,-28.7667,114.6000,71.60,78,0,9.22,1619264286
7,Port Hedland,AU,-20.3167,118.5667,78.80,69,0,4.61,1619264474
8,Kuito,AO,-12.3833,16.9333,75.20,43,0,12.66,1619264256
9,Pedernales,DO,18.0384,-71.7440,79.39,64,0,4.29,1619264496


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# Empty list of hotel at ideal location
hotellist = []

# For loop to find the hotels at the ideal location

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Latitude']
    lng = narrowed_city_df.loc[i]['Longitude']

# setting the parameters
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()

# To find the first hotel from the ideal location
   

    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")

# Add column to the data frame with hotel names.

narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,Cape Town,ZA,-33.9258,18.4232,71.10,56,0,6.91,1619264168,Cape Town
1,São João da Barra,BR,-21.6403,-41.0511,73.40,83,0,4.61,1619264254,Grussaí
2,East London,ZA,-33.0153,27.9116,75.20,53,0,10.36,1619264449,East London
3,Carnarvon,AU,-24.8667,113.6333,75.20,73,0,5.75,1619264253,Carnarvon
4,Arraial do Cabo,BR,-22.9661,-42.0278,74.17,83,0,10.36,1619264326,Arraial do Cabo
5,Tiarei,PF,-17.5333,-149.3333,77.00,78,0,4.61,1619264469,Maha'ena
6,Geraldton,AU,-28.7667,114.6000,71.60,78,0,9.22,1619264286,Geraldton
7,Port Hedland,AU,-20.3167,118.5667,78.80,69,0,4.61,1619264474,Port Hedland
8,Kuito,AO,-12.3833,16.9333,75.20,43,0,12.66,1619264256,Kuito
9,Pedernales,DO,18.0384,-71.7440,79.39,64,0,4.29,1619264496,Anse-à-Pitres


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]

In [38]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))